In [1]:
from langchain.llms import CTransformers
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys

In [2]:
loader = CSVLoader(file_path = "ai_prompts.csv", encoding = "utf-8", csv_args = {'delimiter': ','})
data = loader.load()
data

[Document(page_content='\ufeffPrompts: Get a indexed search result of Manufacturing Item\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/search\nCall: get', metadata={'source': 'ai_prompts.csv', 'row': 0}),
 Document(page_content='\ufeffPrompts: Get a Manufacturing Item\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}\nCall: get', metadata={'source': 'ai_prompts.csv', 'row': 1}),
 Document(page_content='\ufeffPrompts: Modify the Manufacturing Item attributes\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}\nCall: patch', metadata={'source': 'ai_prompts.csv', 'row': 2}),
 Document(page_content='\ufeffPrompts: Delete the Manufacturing Item\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}\nCall: delete', metadata={'source': 'ai_prompts.csv', 'row': 3}),
 Document(page_content='\ufeffPrompts: Get scoped, partial scoped and resulting manufacturing item references from Engineering Item\nPath_Item: /resources/v1/modeler/dsmfg/invoke/dsmfg:getMfgItemsFromEng

In [3]:
# download sentence Transformers embeddings from huggingface

In [4]:
# from langchain_community.embeddings import OllamaEmbeddings
# embeddings = OllamaEmbeddings()

In [5]:
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [6]:
docsearch = FAISS.from_documents(data, embeddings)
docsearch

In [7]:
query = "what is the path_item and call of Gets a indexed search result of Manufacturing Item provided in the data?"

docs = docsearch.similarity_search(query, k =3)
print("Result", docs)

Result [Document(page_content='\ufeffPrompts: Get a indexed search result of Manufacturing Item\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/search\nCall: get', metadata={'source': 'ai_prompts.csv', 'row': 0}), Document(page_content='\ufeffPrompts: Get multiple Manufacturing Items which are Indexed\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/bulkfetch\nCall: post', metadata={'source': 'ai_prompts.csv', 'row': 10}), Document(page_content='\ufeffPrompts: Get Multiple \xa0Manufacturing Item Instances which are Indexed\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItemInstance/bulkFetch\nCall: post', metadata={'source': 'ai_prompts.csv', 'row': 18})]


In [18]:
llm = CTransformers(
    model = "llama-2-7b-chat.ggmlv3.q8_0.bin",
    model_type = "llama",
    config = {"max_new_tokens": 1024, "temperature": 0.015, "context_length" : 2048},
)

In [19]:
template = """
For the given {question}webservice call or {question}combination of two webservice calls give the path_item and call for it in the format : 
"[Path_item : 
Call :
]"
If it requires to do two webservice calls based on the prompt,  give the path_item and call for the first webservice call and the second webservice call in the format :
"[Path_item1 : 
Call1 :
]
[Path_item2 : 
Call2 :
]"
based on the {context}. 

Context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nFor the given {question}webservice call or {question}combination of two webservice calls give the path_item and call for it in the format : \n"[Path_item : \nCall :\n]"\nIf it requires to do two webservice calls based on the prompt,  give the path_item and call for the first webservice call and the second webservice call in the format :\n"[Path_item1 : \nCall1 :\n]\n[Path_item2 : \nCall2 :\n]"\nbased on the {context}. \n\nContext: {context}\n\nQuestion: {question}\n'))]


In [20]:
#Another Template
# template = """ 
# The question contains a webservice call or multiple webservice call prompts,
# If the question has one webservice call Provide the "path_item" and "call" in the format ""[Path_item : 
# Call :
# ]"" based on the context below or If the question has two webservice calls provide "path_item1","call1" and "path_item2","call2" in the format "[Path_item1 : 
#  Call1 :
#  ]
#  [Path_item2 : 
#  Call2 :
#  ]" for it based on the context below

# Context : {context}

# Question: {question}
# """
# prompt = ChatPromptTemplate.from_template(template)

# print(prompt)

In [21]:
parser = StrOutputParser()

In [22]:
# chain = prompt | llm | parser
# chain.input_schema.schema()

In [23]:
retriever = docsearch.as_retriever()

In [24]:
# chain.invoke(
#     {
#         "context": "The name i was given was Monish",
#         "question": "What's my name?"

#     }
# )

In [25]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt | llm | parser
)

chain.invoke({"question": "Modifies the Manufacturing Item Instance attributes"})

'Path_item : /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{PID}/dsmfg:MfgItemInstance/{ID}\nCall : patch\n\nAnswer: The path item is /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{PID}/dsmfg:MfgItemInstance/{ID} and the call is patch.'

In [26]:
# chain = (
#     {
#         "context": itemgetter("question") | retriever,
#         "question": itemgetter("question")
#     }
#     | prompt | llm | parser
# )

# chain.invoke({"question": "Create and Delete Manufacturing Items"})

In [ ]:
# qa = ConversationalRetrievalChain.from_llm(llm, retriever = docsearch.as_retriever())

while True:
    chat_history = []

    query = input(f"Input Prompt: ")
    if query == 'exit':
    
        print('Exiting')
        sys.exit()

    if query == '':
        continue
    result = chain.invoke({"question": query, "chat_history": chat_history})
    print("Response: ", result)

Input Prompt:  Modify Configuration Information of configured object and Get a Object Configuration information


Response:  
Answer: Based on the provided prompts, the path item and call for the combination of two webservice calls would be :
"[Path_item1 : /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured\nCall1 : patch]
[Path_item2 : /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured/get\nCall2 : get]"

Explanation:
The first webservice call is "Modify Configuration Information of configured object" which has the path item as "/resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured" and the call as "patch".
The second webservice call is "Get a Object Configuration information" which has the path item as "/resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured/get" and the call as "get".
By combining these two webservice calls, we can modify the configuration information of a configured object and then get the updated configuration information.


Input Prompt:  Create Manufacturing Item and Modify the Manufacturing Item attributes


Response:  Please provide the path item and call for the webservice calls based on the prompts provided.

Answer: Based on the prompts provided, the path items and calls for the two webservice calls are as follows:

Path Item 1 : /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{PID}/dsmfg:MfgItemInstance/{ID}
Call : patch

Path Item 2 : /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}
Call : patch

Explanation:
The first webservice call is to modify the attributes of a specific manufacturing item instance, and the path item includes the ID of the manufacturing item instance. The call is patch, which means that only the fields specified in the request body will be updated, and any unspecified fields will retain their current values.

The second webservice call is to modify the attributes of a specific manufacturing item, and the path item includes the ID of the manufacturing item. Again, the call is patch, which means that only the fields specified in the request body will be updated, and any u

Input Prompt:  Create Manufacturing Item Instance and Replace the Manufacturing Item Instance


Response:  
Answer: Based on the provided prompts, the path item and call for creating a manufacturing item instance and replacing an existing one are as follows:

Path Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{PID}/dsmfg:MfgItemInstance/{ID}/replace
Call: post

Explanation: The path item is the URL of the API endpoint for creating a new manufacturing item instance and replacing an existing one. The call is the HTTP method used to make the request (in this case, POST).


Input Prompt:  Update Configuration of configured object and fetch Object Configuration 


Response:  
Answer: Based on the provided prompts, the path item and call for the combination of two webservice calls are:

Path Item : /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured
Call : patch (for updating configuration information) followed by get (for fetching object configuration)

Therefore, the path item and call for the combination of two webservice calls are:
"[Path_item : /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured]
[Call : patch, get]"


Input Prompt:  who is newton


Response:  
Answer: Based on the provided documents, it appears that "newton" refers to a web service call in the context of the API for the Digital Manufacturing Framework (DMF). Specifically, it seems to be related to the "dsmfg" namespace.

Here are the path items and calls associated with each document:

* Document(page_content='\ufeffPrompts: Modify Configuration Information of configured object\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured\nCall: patch', metadata={'source': 'ai_prompts.csv', 'row': 49}):
	+ Path item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dscfg:Configured
	+ Call: patch
* Document(page_content='\ufeffPrompts: Get a Change Control of an dsmfg:MfgItem\nPath_Item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dslc:changeControl\nCall: get', metadata={'source': 'ai_prompts.csv', 'row': 47}):
	+ Path item: /resources/v1/modeler/dsmfg/dsmfg:MfgItem/{ID}/dslc:changeControl
	+ Call: get
* Document(page_content='\ufeffPrompts: Get a

In [ ]:
#  qa = ConversationalRetrievalChain.from_llm(llm, retriever = docsearch.as_retriever())

# while True:
#     chat_history = []

#     query = input(f"Input Prompt: ")
#     if query == 'exit':
#         print('Exiting')
#         sys.exit()

#     if query == '':
#         continue
#     result = qa.invoke({"question": query, "chat_history": chat_history})
#     print("Response: ", result['answer'])